In [15]:
pip install rdkit-pypi

In [16]:
import pandas as pd
import rdkit
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem, MACCSkeys, RDKFingerprint
from rdkit.Avalon.pyAvalonTools import GetAvalonFP
from padelpy import padeldescriptor
from pathlib import Path

In [5]:
df = pd.read_csv('clean_activity.csv')

In [6]:
df

,Molecule ChEMBL ID,Molecule Name,Molecular Weight,Smiles,AlogP,Ligand Efficiency BEI,Ligand Efficiency LE,Ligand Efficiency LLE,Ligand Efficiency SEI,Standard Type,...,Standard Value,Standard Units,pChEMBL Value,Target ChEMBL ID,Target Name,Target Organism,Action Type,AlogP.1,Molecular Weight.1,Ligand Efficiency LE.1
0,CHEMBL100091,NaN,457.95,CCCCOCCOC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc(...,5.20,15.77,0.31,2.02,10.82,IC50,...,60.0,nM,7.22,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,5.20,457.95,0.31
1,CHEMBL100092,NaN,441.96,CCCCCCOC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc(O...,5.97,16.34,0.32,1.25,12.55,IC50,...,60.0,nM,7.22,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,5.97,441.96,0.32
2,CHEMBL100156,NaN,314.43,CS(=O)(=O)c1ccc(-c2cscc2-c2ccccc2)cc1,4.49,17.17,0.35,0.91,15.81,IC50,...,4000.0,nM,5.40,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,4.49,314.43,0.35
3,CHEMBL100250,NaN,314.43,CS(=O)(=O)c1ccc(-c2ccsc2-c2ccccc2)cc1,4.49,17.17,0.35,0.91,15.81,IC50,...,4000.0,nM,5.40,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,4.49,314.43,0.35
4,CHEMBL100331,NaN,469.03,CCCCCCCCNC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc...,6.32,15.77,0.31,1.08,12.26,IC50,...,40.0,nM,7.40,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,6.32,469.03,0.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,CHEMBL99374,NaN,328.46,Cc1ccc(-c2cscc2-c2ccc(S(C)(=O)=O)cc2)cc1,4.79,16.43,0.34,0.61,15.81,IC50,...,4000.0,nM,5.40,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,4.79,328.46,0.34
5096,CHEMBL99517,NaN,328.46,Cc1ccc(-c2sccc2-c2ccc(S(C)(=O)=O)cc2)cc1,4.79,14.88,0.30,0.10,14.31,IC50,...,13000.0,nM,4.89,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,4.79,328.46,0.30
5097,CHEMBL99727,NaN,432.54,CCOC(=O)CCc1cc(-c2ccc(S(C)(=O)=O)cc2)c(-c2ccc(...,5.12,NaN,NaN,NaN,NaN,IC50,...,130000.0,nM,NaN,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,5.12,432.54,NaN
5098,CHEMBL99828,NaN,346.45,Cc1cc(-c2ccc(S(C)(=O)=O)cc2)c(-c2ccc(F)cc2)s1,4.93,14.43,0.30,0.07,14.65,IC50,...,10000.0,nM,5.00,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,4.93,346.45,0.30


2,3 Рассчитаем дескрипторы и фингерпринты:

In [7]:
df[['Smiles', 'Molecule ChEMBL ID']].to_csv("molecules.smi", sep=" ", index=False, header=False)


In [8]:
df_smi = pd.read_csv("molecules.smi", sep=" ", names=["Smiles", "ID"])

batch_size = 500


for i, chunk in enumerate(np.array_split(df_smi, len(df_smi) // batch_size + 1)):
    chunk.to_csv(f"batch_{i}.smi", sep=" ", index=False, header=False)

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [9]:
for i in range(0, 11):
    input_file = f"batch_{i}.smi"
    output_file = f"descriptors_batch_{i}.csv"

    padeldescriptor(
        mol_dir=input_file,
        d_file=output_file,
        fingerprints=True,
        d_2d=True,
        d_3d=False,
        removesalt=True,
        log=True,
        threads=4,
        maxruntime=30000,
        retainorder=True,
        standardizetautomers=True,
    )

In [10]:
import glob

all_batches = glob.glob("descriptors_batch_*.csv")
df_all = pd.concat([pd.read_csv(f) for f in all_batches], ignore_index=True)

df_all.to_csv("padel_descriptors_all.csv", index=False)

In [11]:
df_descriptors= pd.read_csv("padel_descriptors_all.csv")

In [12]:
df

,Molecule ChEMBL ID,Molecule Name,Molecular Weight,Smiles,AlogP,Ligand Efficiency BEI,Ligand Efficiency LE,Ligand Efficiency LLE,Ligand Efficiency SEI,Standard Type,...,Standard Value,Standard Units,pChEMBL Value,Target ChEMBL ID,Target Name,Target Organism,Action Type,AlogP.1,Molecular Weight.1,Ligand Efficiency LE.1
0,CHEMBL100091,NaN,457.95,CCCCOCCOC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc(...,5.20,15.77,0.31,2.02,10.82,IC50,...,60.0,nM,7.22,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,5.20,457.95,0.31
1,CHEMBL100092,NaN,441.96,CCCCCCOC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc(O...,5.97,16.34,0.32,1.25,12.55,IC50,...,60.0,nM,7.22,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,5.97,441.96,0.32
2,CHEMBL100156,NaN,314.43,CS(=O)(=O)c1ccc(-c2cscc2-c2ccccc2)cc1,4.49,17.17,0.35,0.91,15.81,IC50,...,4000.0,nM,5.40,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,4.49,314.43,0.35
3,CHEMBL100250,NaN,314.43,CS(=O)(=O)c1ccc(-c2ccsc2-c2ccccc2)cc1,4.49,17.17,0.35,0.91,15.81,IC50,...,4000.0,nM,5.40,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,4.49,314.43,0.35
4,CHEMBL100331,NaN,469.03,CCCCCCCCNC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc...,6.32,15.77,0.31,1.08,12.26,IC50,...,40.0,nM,7.40,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,6.32,469.03,0.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,CHEMBL99374,NaN,328.46,Cc1ccc(-c2cscc2-c2ccc(S(C)(=O)=O)cc2)cc1,4.79,16.43,0.34,0.61,15.81,IC50,...,4000.0,nM,5.40,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,4.79,328.46,0.34
5096,CHEMBL99517,NaN,328.46,Cc1ccc(-c2sccc2-c2ccc(S(C)(=O)=O)cc2)cc1,4.79,14.88,0.30,0.10,14.31,IC50,...,13000.0,nM,4.89,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,4.79,328.46,0.30
5097,CHEMBL99727,NaN,432.54,CCOC(=O)CCc1cc(-c2ccc(S(C)(=O)=O)cc2)c(-c2ccc(...,5.12,NaN,NaN,NaN,NaN,IC50,...,130000.0,nM,NaN,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,5.12,432.54,NaN
5098,CHEMBL99828,NaN,346.45,Cc1cc(-c2ccc(S(C)(=O)=O)cc2)c(-c2ccc(F)cc2)s1,4.93,14.43,0.30,0.07,14.65,IC50,...,10000.0,nM,5.00,CHEMBL230,Cyclooxygenase-2,Homo sapiens,NaN,4.93,346.45,0.30


In [13]:
df = pd.concat([df, df_descriptors], axis=1)

In [17]:
df

,Molecule ChEMBL ID,Molecule Name,Molecular Weight,Smiles,AlogP,Ligand Efficiency BEI,Ligand Efficiency LE,Ligand Efficiency LLE,Ligand Efficiency SEI,Standard Type,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,CHEMBL100091,NaN,457.95,CCCCOCCOC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc(...,5.20,15.77,0.31,2.02,10.82,IC50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CHEMBL100092,NaN,441.96,CCCCCCOC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc(O...,5.97,16.34,0.32,1.25,12.55,IC50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,CHEMBL100156,NaN,314.43,CS(=O)(=O)c1ccc(-c2cscc2-c2ccccc2)cc1,4.49,17.17,0.35,0.91,15.81,IC50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CHEMBL100250,NaN,314.43,CS(=O)(=O)c1ccc(-c2ccsc2-c2ccccc2)cc1,4.49,17.17,0.35,0.91,15.81,IC50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CHEMBL100331,NaN,469.03,CCCCCCCCNC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc...,6.32,15.77,0.31,1.08,12.26,IC50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,CHEMBL99374,NaN,328.46,Cc1ccc(-c2cscc2-c2ccc(S(C)(=O)=O)cc2)cc1,4.79,16.43,0.34,0.61,15.81,IC50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5096,CHEMBL99517,NaN,328.46,Cc1ccc(-c2sccc2-c2ccc(S(C)(=O)=O)cc2)cc1,4.79,14.88,0.30,0.10,14.31,IC50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5097,CHEMBL99727,NaN,432.54,CCOC(=O)CCc1cc(-c2ccc(S(C)(=O)=O)cc2)c(-c2ccc(...,5.12,NaN,NaN,NaN,NaN,IC50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5098,CHEMBL99828,NaN,346.45,Cc1cc(-c2ccc(S(C)(=O)=O)cc2)c(-c2ccc(F)cc2)s1,4.93,14.43,0.30,0.07,14.65,IC50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
def get_fingerprints(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None, None, None

    maccs_fp = MACCSkeys.GenMACCSKeys(mol)

    return maccs_fp


In [19]:
maccs_fps = df["Smiles"].apply(get_fingerprints)

In [20]:
maccs_fps

,Smiles
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...
5095,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5096,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5097,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5098,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [21]:
def fp_to_array(fp):
    return list(fp.ToBitString()) if fp is not None else [np.nan]

In [22]:
df_maccs = pd.DataFrame([fp_to_array(fp) for fp in maccs_fps])
df_maccs = df_maccs.fillna(0).astype(int)

In [23]:
df_maccs

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,1,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,1,1,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,1,1,1,1,0
5096,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,1,1,1,1,0
5097,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,1,1,1,1,0
5098,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,1,1,1,1,0


4. Отфильтруем и отберем признаки:

In [24]:
df = df.loc[:, df.nunique() > 1]

In [25]:
df

,Molecule ChEMBL ID,Molecule Name,Molecular Weight,Smiles,AlogP,Ligand Efficiency BEI,Ligand Efficiency LE,Ligand Efficiency LLE,Ligand Efficiency SEI,Standard Relation,...,PubchemFP835,PubchemFP836,PubchemFP839,PubchemFP840,PubchemFP845,PubchemFP860,PubchemFP861,PubchemFP863,PubchemFP866,PubchemFP868
0,CHEMBL100091,NaN,457.95,CCCCOCCOC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc(...,5.20,15.77,0.31,2.02,10.82,'=',...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CHEMBL100092,NaN,441.96,CCCCCCOC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc(O...,5.97,16.34,0.32,1.25,12.55,'=',...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,CHEMBL100156,NaN,314.43,CS(=O)(=O)c1ccc(-c2cscc2-c2ccccc2)cc1,4.49,17.17,0.35,0.91,15.81,'=',...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CHEMBL100250,NaN,314.43,CS(=O)(=O)c1ccc(-c2ccsc2-c2ccccc2)cc1,4.49,17.17,0.35,0.91,15.81,'=',...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CHEMBL100331,NaN,469.03,CCCCCCCCNC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc...,6.32,15.77,0.31,1.08,12.26,'=',...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,CHEMBL99374,NaN,328.46,Cc1ccc(-c2cscc2-c2ccc(S(C)(=O)=O)cc2)cc1,4.79,16.43,0.34,0.61,15.81,'=',...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5096,CHEMBL99517,NaN,328.46,Cc1ccc(-c2sccc2-c2ccc(S(C)(=O)=O)cc2)cc1,4.79,14.88,0.30,0.10,14.31,'=',...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5097,CHEMBL99727,NaN,432.54,CCOC(=O)CCc1cc(-c2ccc(S(C)(=O)=O)cc2)c(-c2ccc(...,5.12,NaN,NaN,NaN,NaN,'=',...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5098,CHEMBL99828,NaN,346.45,Cc1cc(-c2ccc(S(C)(=O)=O)cc2)c(-c2ccc(F)cc2)s1,4.93,14.43,0.30,0.07,14.65,'=',...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
numeric_df = df.select_dtypes(include=[np.number])

corr_matrix = numeric_df.corr().abs()


upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.7)]


df = df.drop(columns=to_drop)

In [27]:
df = df.select_dtypes(exclude=["object", "bool"])

In [32]:
df

,Molecular Weight,AlogP,Ligand Efficiency BEI,Ligand Efficiency SEI,Standard Value,pChEMBL Value,nAcid,ALogP,ALogp2,naAromAtom,...,PubchemFP770,PubchemFP771,PubchemFP772,PubchemFP839,PubchemFP840,PubchemFP845,PubchemFP860,PubchemFP861,PubchemFP863,PubchemFP868
0,457.95,5.20,15.77,10.82,60.0,7.22,0,-0.2031,0.041250,15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,441.96,5.97,16.34,12.55,60.0,7.22,0,-0.6482,0.420163,15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,314.43,4.49,17.17,15.81,4000.0,5.40,0,0.2677,0.071663,17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,314.43,4.49,17.17,15.81,4000.0,5.40,0,0.2677,0.071663,17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,469.03,6.32,15.77,12.26,40.0,7.40,0,-1.7446,3.043629,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,328.46,4.79,16.43,15.81,4000.0,5.40,0,-1.1828,1.399016,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5096,328.46,4.79,14.88,14.31,13000.0,4.89,2,1.0876,1.182874,17,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5097,432.54,5.12,NaN,NaN,130000.0,NaN,0,0.7624,0.581254,15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5098,346.45,4.93,14.43,14.65,10000.0,5.00,0,0.8334,0.694556,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
df.to_csv("filtered_descriptors.csv", index=False)
